In [1]:
import numpy as np 
import pandas as pd 
import warnings;warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,Flatten
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('train.csv',lineterminator='\n')
df_test = pd.read_csv('test.csv',lineterminator='\n')

In [3]:
df_train['label'] = df_train['label'].map({'Negative':0,'Positive':1})
df_train.head(20)

,ID,review,label
0,1,Jo bhi ap se tou behtar hoon,0
1,2,ya Allah meri sister Affia ki madad farma,1
2,3,Yeh khud chahta a is umar main shadi krna. ha...,0
3,4,Tc ? Apky mun xe exe alfax achy nae lgty 😒💃,0
4,5,Good,1
5,6,American president John f Kennedy aur in ke bh...,0
6,7,"Commission aur kickback ka dor Dora raha, quo...",0
7,8,Allah pak nazer e bd sy bechye or humesha boha...,1
8,9,Amoman log samajhte hain ke jhok siyal hi Abid...,1
9,10,Akki KhanYani k tum ....... v good Wesy tum sh...,0


In [4]:
#test if nan exists
df_train.isnull().sum()

ID        0
review    0
label     0
dtype: int64

In [5]:
df_test.isnull().sum()

ID        0
review    0
dtype: int64

In [6]:
df_test.head()

,ID,review
0,1,Yaqoob Memon Ki Phansi Zalimana Ghair Insani H...
1,2,Sabit qadam rehna
2,3,Good decision on ko bhi aam shehryun ki tarah ...
3,4,Jo Shakhs ALLAH Aur Qayamat Per Eman Rakhta Ho...
4,5,Tm log veriat kohli ko ahmad shahzad k sath co...


In [7]:
df_test.iloc[2711]

ID                                                     2712
review    Mery liye dua kry Allah pak naseeb achy kry ameen
Name: 2711, dtype: object

In [8]:
numpy_array = df_train.as_matrix()
numpy_array_test = df_test.as_matrix()

In [9]:
#two commom ways to clean data
def cleaner(word):
  word = re.sub(r'\#\.', '', word)
  word = re.sub(r'\n', '', word)
  word = re.sub(r',', '', word)
  word = re.sub(r'\-', ' ', word)
  word = re.sub(r'\.', '', word)
  word = re.sub(r'\\', ' ', word)
  word = re.sub(r'\\x\.+', '', word)
  word = re.sub(r'\d', '', word)
  word = re.sub(r'^_.', '', word)
  word = re.sub(r'_', ' ', word)
  word = re.sub(r'^ ', '', word)
  word = re.sub(r' $', '', word)
  word = re.sub(r'\?', '', word)

  return word.lower()
def hashing(word):
  word = re.sub(r'ain$', r'ein', word)
  word = re.sub(r'ai', r'ae', word)
  word = re.sub(r'ay$', r'e', word)
  word = re.sub(r'ey$', r'e', word)
  word = re.sub(r'ie$', r'y', word)
  word = re.sub(r'^es', r'is', word)
  word = re.sub(r'a+', r'a', word)
  word = re.sub(r'j+', r'j', word)
  word = re.sub(r'd+', r'd', word)
  word = re.sub(r'u', r'o', word)
  word = re.sub(r'o+', r'o', word)
  word = re.sub(r'ee+', r'i', word)
  if not re.match(r'ar', word):
    word = re.sub(r'ar', r'r', word)
  word = re.sub(r'iy+', r'i', word)
  word = re.sub(r'ih+', r'eh', word)
  word = re.sub(r's+', r's', word)
  if re.search(r'[rst]y', 'word') and word[-1] != 'y':
    word = re.sub(r'y', r'i', word)
  if re.search(r'[bcdefghijklmnopqrtuvwxyz]i', word):
    word = re.sub(r'i$', r'y', word)
  if re.search(r'[acefghijlmnoqrstuvwxyz]h', word):
    word = re.sub(r'h', '', word)
  word = re.sub(r'k', r'q', word)
  return word

def array_cleaner(array):
  # X = array
  X = []
  for sentence in array:
    clean_sentence = ''
    words = sentence.split(' ')
    for word in words:
      clean_sentence = clean_sentence +' '+ cleaner(word)
    X.append(clean_sentence)
  return X

In [10]:
X_test = numpy_array_test[:,1]
X_train = numpy_array[:, 1]
# Clean X here
X_train = array_cleaner(X_train)
X_test = array_cleaner(X_test)
y_train = numpy_array[:, 2]

In [11]:
print(len(X_train))
print(len(X_test))
print(len(y_train))

6328
2712
6328


In [12]:
y_train = np.array(y_train)
y_train = y_train.astype('int8')
y_train[:6]

array([0, 1, 0, 0, 1, 0], dtype=int8)

In [13]:
X_all = X_train + X_test # Combine both to fit the tokenizer.
lentrain = len(X_train)

In [14]:
tokenizer = Tokenizer(
    nb_words=2000, 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(X_all)

In [15]:
X_all[:2]

[' jo bhi ap se tou behtar hoon', ' ya allah meri sister affia ki madad farma']

In [16]:
X = tokenizer.texts_to_sequences(X_all)
# X = pad_sequences(X)
X[:2]

[[32, 11, 35, 8, 98, 284, 268], [51, 16, 87, 1, 252, 363]]

In [17]:
X = pad_sequences(X)
X[:2]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [18]:
print(X.shape)

(9040, 219)


In [19]:
len(X)

9040

In [20]:
embed_dim = 128
lstm_out = 256
batch_size = 32

model = Sequential()
model.add(Embedding(2000,embed_dim, input_length=X.shape[1],dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2,return_sequences=True))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 219, 128)          256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 219, 256)          394240    
_________________________________________________________________
flatten_1 (Flatten)          (None, 56064)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 112130    
Total params: 762,370
Trainable params: 762,370
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
X_train = X[:lentrain] # Separate back into training and test sets. 
X_test = X[lentrain:]

In [22]:
y_binary = to_categorical(y_train)

In [23]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val,self.y_val = validation_data
    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print('\n ROC_AUC - epoch:%d - score:%.6f \n' % (epoch+1, score))
#             print(y_pred[:6])
x_train5,y_train5,x_label5,y_label5 = train_test_split(X_train,y_binary, train_size=0.8, random_state=234)
RocAuc = RocAucEvaluation(validation_data=(y_train5,y_label5), interval=1)

In [24]:
hist = model.fit(x_train5, x_label5, batch_size=batch_size, epochs=1, validation_data=(y_train5, y_label5), callbacks=[RocAuc], verbose=2)

Train on 5062 samples, validate on 1266 samples
Epoch 1/1
 - 140s - loss: 0.6301 - acc: 0.6213 - val_loss: 0.5292 - val_acc: 0.7425

 ROC_AUC - epoch:1 - score:0.813401 



In [25]:
y_lstm = model.predict_proba(X_test,batch_size=batch_size)[:,1]

In [26]:
print(len(y_lstm))
y_lstm[:7]


2712


array([0.4398137 , 0.5837693 , 0.7322361 , 0.69807845, 0.23353294,
       0.8086587 , 0.18370664], dtype=float32)

In [27]:
lstm_output = pd.DataFrame(data={"ID":df_test["ID"], "Pred":y_lstm})
lstm_output.to_csv('lstm_new.csv', index = False, quoting = 3)